In [1]:
import os
from glob import glob

import torch
import torch.backends.cudnn as cudnn
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import torchvision.datasets as dset
from torch.utils.data import DataLoader
from torchvision import transforms
from configurate import get_config
from protonets import ProtoNet
from utils import AverageMeter
from tqdm.notebook import tqdm

from dataloader_cifar import PrototypicalBatchSampler

from easydict import EasyDict

In [2]:
train_dataset = dset.CIFAR100('./data/cifar100', train=True, download=True, transform=transforms.ToTensor())
test_dataset = dset.CIFAR100('./data/cifar100', train=False, download=False, transform=None)

Files already downloaded and verified


In [3]:
labels = []
for data in train_dataset:
    labels.append(data[1])


In [8]:
sampler = PrototypicalBatchSampler(torch.LongTensor(labels), 100, 250,250, 100)

data_loader = DataLoader(train_dataset, batch_sampler=sampler)

In [9]:
data = next(iter(data_loader))

x,y = data[0], data[1]

50000
50000
100
tensor([250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250,
        250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250,
        250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250,
        250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250,
        250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250,
        250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250,
        250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250,
        250, 250])
